# iSEPTA Philly Project
## "The Tappiest Time of the Year" Contest

### by John A. Fonte

__Goal:__ Guess the total number of SEPTA key taps done on Friday, December 20, 2019 _without going over._

- __Definition:__ A SEPTA "key tap" is a payment for public transportation fare by way of "tapping" the recently introduced magnetic cards onto digital turnstiles. <br><br>
  - For purposes of this project, a "key tap" shall refer to a single use of a SEPTA key card for payment of fare.

__Link to Contest:__ https://www.iseptaphilly.com/contests/172

   - (_Update:_ The contest closed on December 20, 2019.)

***

***
# OVERVIEW OF PROJECT

We need to perform the following for our analysis:

1. Figure out what data to obtain.
2. Where to go to obtain said data.
3. Within the desired data, figure out what metric to use to determine SEPTA key taps.
4. Perform data analysis and data modeling, assuming that there is enough available relevant data.
5. Draw Conclusions

## 1. What Data includes SEPTA Key Card Taps?

SEPTA key taps are done for fare on all four modes of Philadelphia public transportation: 

   1. Subway (Market-Frankford Line (MFL) and Broad Street Line (BSL)) 
   2. Buses
   3. Trolleys 
   4. Regional Rail Lines
   
SEPTA key cards were introduced tpo the public on February 9, 2017, according to the timeline set forth on the [Wikipedia page](https://en.wikipedia.org/wiki/SEPTA_Key).
   
### Data Limitation #1: No Historic Key Card Information for Regional Rail Lines

The use of SEPTA key cards is a recent change to Philadelphia's public transportation system, and indeed, SEPTA is still in the process of transitioning. Notably, the transition from standard payment to key taps for regional rail lines is [still in flux](https://www.inquirer.com/transportation/septa-key-regional-rail-rollout-travel-wallet-20191118.html). It is unclear when the transition will be completed; according to the Wikipedia timeline, as of now, the regional rail transition is still in its early adoption program phase.

As such, there is not sufficient information to determine what portion of regional rail riders are currently using the SEPTA key card as the transition is so new (and quite frankly, it is unclear if this is calculated into the total number on iSEPTA Philly's end).

For this reason, we will not include regional rail ridership into the analysis.

### Data Limitation #2: Fares Paid without SEPTA Key Card

Although the SEPTA key card has made fare tokens obsolete, individuals may still pay for fare via single "quick trip" purchases at digital kiosks or with cash via a SEPTA representative in person. As we will see, the data does not show a distinction between key tap ridership and non-key tap ridership.
***

***
## 2. Where to Find SEPTA Data

Luckily, Philadelphia is fairly progressive in offering open data. Here are some open data links used to obtain the data:

1. __[SEPTA Open Data](http://septaopendata-septa.opendata.arcgis.com/):__ Provides .csv compilations of SEPTA statistics for each mode of transportation described above, along with aggregations of data.


2. __Federal Transit Adminsitration's [National Transit Database](https://www.transit.dot.gov/ntd/what-national-transit-database-ntd-program):__ Has reports on all major public transportation systems, in both aggregated and raw data form.


3. __Various Aggregated Reports:__ These reports do not provide analyzable tabular data, but they are still good reference points/sanity checks for a ballpark estimation as to where are analyzed values should be.

    - _SEPTA Annual Reports_: Annual Reports provided by SEPTA itself, such as [this one for November 2019](http://septa.org/strategic-plan/pdf/2019-11-revenue-ride.pdf)!<br><br>
    
    - _Center City District & Central Philadelphia Development Corporation_: Has Annual Reports such as this [2018 Annual Report](https://centercityphila.org/uploads/attachments/cjusnd2j40fjnukqdy7j9p2ng-socc-2019-transportation.pdf).
    

Here are some open data links that seemed relevant but are __NOT__ used in this project:

1. __[Scraped Septa-stats.com Data](https://www.dropbox.com/sh/3jnvonaqtmvc3wh/AACvwz3DMTXrW56P8xBUUIcSa?dl=0):__ This is voluminous data on each and every SEPTA service route and timestamps for each stop on each and every service route. This data does not include ridership or revenue stats - only the routes themselves.
    - Plus...I am pretty sure the .json format is broken because a parsing error is thrown on the first line! I think there is no delimiter (i.e., comma) between json lines, and that is simply not worth fixing, especially for irrelevant data.
***

### 2.A. DATA WRANGLING: Importing and Loading the Raw Data

The main source of data was Spring 2018 pre-aggregated datasets for each mode of transportation (bus, trolley, rail, etc.) from [SEPTA OpenData ArcGIS Philadelphia](http://septaopendata-septa.opendata.arcgis.com/). Other aggregated datasets existed for these relevant modes of transportation, but those seemed to overlap with the ones chosen here. 

Accordingly, I am using the available .csv datasets that include route ridership _and_ revenue information from Fiscal Year 2017.

In [1]:
import pandas as pd

In [2]:
'''
Data obtained from SEPTA Open Data and
the FTA National Transit Database. These
data files were downloaded onto local terminal.
'''
# (local path reads)

## Subway Aggregated Datasets - Spring 2018 (with revenue of Fiscal Year 2017)

df_BSLagg = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA__Broad_Street_Line.csv')
df_MFLagg = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA_Market_Frankford_Line.csv')
df_Norrisagg = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA__Norristown_Highspeed_Line - agg.csv')

In [3]:
## Bus & Trolley Aggregated Datasets - Spring 2018

df_trolleyagg = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA_Trolley - Spring 2018 revenue.csv')
df_busagg = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA_Bus - Spring 2018 revenue.csv')

***
### 2.B. DATA CLEANING: Creating the Main Datasets

In [6]:
# Need to clean every file

## Example of what we're dealing with:
df_BSLagg.head()

,OBJECTID,Route,Vehicle_Hours,Vehicle_Miles,Peak_Vehicles,Average_Weekday_Passengers,Annual_Passengers,Average_Trip_Length,Passenger_Miles,Passenger_Revenue,Variable_Expenses,Variable_Recovery,Fully_Allocated_Expenses,Operating_Ratio,GlobalID,Shape__Length
0,1,Broad Street Line,395645,6805092,105,111575,33850366,4,135401500,36968030,28995335,127,101140835,37,1b333479-b0ba-4fe9-98ef-60891c620fdf,0.173819


In [7]:
df_Norrisagg.head()

,OBJECTID,Route,Vehicle_Hours,Vehicle_Miles,Peak_Vehicles,Average_Weekday_Passengers,Annual_Passengers,Average_Trip_Length,Passenger_Miles,Passenger_Revenue,Variable_Expenses,Variable_Recovery,Fully_Allocated_Expenses,Operating_Ratio,Miles,GlobalID,Shape__Length
0,1,Norristown Highspeed Line,49196,934731,16,10525,3106320,9.2,28578100,3799029,6056295,63,12515495,30,13.35,402ee12a-b566-4e20-94ed-a0c886421534,0.214819


In [8]:
len(df_BSLagg.columns)

16

In [9]:
df_subwayAgg = pd.concat([df_BSLagg, df_MFLagg, df_Norrisagg], sort=False)
df_subwayAgg

,OBJECTID,Route,Vehicle_Hours,Vehicle_Miles,Peak_Vehicles,Average_Weekday_Passengers,Annual_Passengers,Average_Trip_Length,Passenger_Miles,Passenger_Revenue,Variable_Expenses,Variable_Recovery,Fully_Allocated_Expenses,Operating_Ratio,GlobalID,Shape__Length,Miles
0,1,Broad Street Line,395645,6805092,105,111575,33850366,4.0,135401500,36968030,28995335,127,101140835,37,1b333479-b0ba-4fe9-98ef-60891c620fdf,0.173819,NaN
0,1,Market-Frankford Line,470930,9230221,144,180512,54767414,4.7,257406800,59810392,137752084,158,136694484,44,9b7ec014-18d2-42af-a482-a86be8a370e6,0.237995,NaN
0,1,Norristown Highspeed Line,49196,934731,16,10525,3106320,9.2,28578100,3799029,6056295,63,12515495,30,402ee12a-b566-4e20-94ed-a0c886421534,0.214819,13.35


In [10]:
# dropping irrelevant columns
'''
Because more to drop than to keep, 
it's easier to identify/list the columns 
that COULD be helpful features!
'''

df_subwayAgg = df_subwayAgg[['Route', 'Average_Weekday_Passengers', 'Annual_Passengers',
                            'Average_Trip_Length', 'Passenger_Miles', 'Passenger_Revenue', 
                            'Operating_Ratio']].copy()

In [12]:
# Trolley Agg 
'''
Checking what additional features, if any, 
would possibly pose as a benefit to our data analysis.
Upon looking...only 'School Route' jumps out at me.
'''
df_trolleyagg.columns

Index(['OBJECTID', 'Route', 'Route_Name', 'District_1', 'District_2',
       'District_3', 'District_4', 'Revenue', 'Mode', 'Vehicle_Group',
       'Vehicle_Hours', 'Vehicle_Miles', 'Peak_Vehicles',
       'Average_Weekday_Passengers', 'Annual_Passengers',
       'Average_Trip_Length', 'Passenger_Miles', 'Passenger_Revenue',
       'Variable_Expenses', 'Variable_Recovery', 'Fully_Allocated_Expenses',
       'Operating_Ratio', 'Comments', 'School_Route', 'GlobalID',
       'Shape__Length'],
      dtype='object')

In [45]:
df_trolleyagg.School_Route.value_counts()
# ...except, not very helpful...

No    8
Name: School_Route, dtype: int64

In [13]:
# adding this to subwayAgg - lining up "route" and "vehicle" accordingly

df_subwayAgg['Vehicle_Group'] = 'Subway' # "vectored operation" - applies to entire column

In [17]:
df_trolleyagg = df_trolleyagg[['Route', 'Average_Weekday_Passengers', 'Annual_Passengers',
                            'Average_Trip_Length', 'Passenger_Miles', 'Passenger_Revenue', 
                            'Operating_Ratio', 'Vehicle_Group']].copy()

In [53]:
#combining agg df's

df_aggregate = pd.concat([df_subwayAgg, df_trolleyagg], sort=False)

In [54]:
# rearranging columns to what I think makes sense

df_aggregate = df_aggregate[['Vehicle_Group','Route', 'Average_Trip_Length', 'Operating_Ratio',
                             'Annual_Passengers', 'Average_Weekday_Passengers',
                             'Passenger_Miles', 'Passenger_Revenue']].copy()

In [55]:
df_aggregate.reset_index(drop=True)

,Vehicle_Group,Route,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
0,Subway,Broad Street Line,4.0,37,33850366.00,111575,135401500,36968030
1,Subway,Market-Frankford Line,4.7,44,54767414.00,180512,257406800,59810392
2,Subway,Norristown Highspeed Line,9.2,30,3106320.00,10525,28578100,3799029
3,Trolley,11,2.6,39,4586909.00,14822,11926000,4952623
4,Trolley,13,2.8,34,4491464.00,14512,12576100,4849568
5,Trolley,34,2.2,30,4246669.00,13723,9342700,4585256
6,Trolley,36,2.6,30,4458107.00,14406,11591100,4813552
7,Trolley,10,1.9,34,4223128.00,13645,8023900,4559838
8,Trolley,15,1.8,31,3140497.00,10147,5652900,3390889
9,Trolley,101,4.1,20,999060.00,4235,4096100,1248720


In [56]:
df_aggregate.rename(columns={'Vehicle_Group':'Vehicle_Type'}, inplace=True)

In [38]:
df_busagg.rename(columns={'Mode':'Vehicle_Type'}, inplace=True)

In [58]:
# combining bus_agg to df_aggregate
# miscellaneous rearrangements of columns and column renames for aligned concatenation

df_busagg = df_busagg[['Vehicle_Type', 'Route', 'School_Route', 'Average_Trip_Length', 
                       'Operating_Ratio', 'Annual_Passengers', 
                       'Average_Weekday_Passengers', 'Passenger_Miles', 'Passenger_Revenue']].copy()

In [59]:
df_aggregate['School_Route'] = 'No'

In [61]:
df_aggregate1 = pd.concat([df_busagg, df_aggregate], sort=False, ignore_index=True) 

# when sort=False, column arrangement is assigned by first concat df value
# assigned new variable so I don't have to go all the way back in case I irrevocably screw up
## (you always learn this the hard way...)

In [62]:
# Checking for NaN's - simple loop

for column in df_aggregate1.columns:
    print("The number of NaN's in the column {} is {}.".format(column, df_aggregate1[column].isna().sum()))

The number of NaN's in the column Vehicle_Type is 0.
The number of NaN's in the column Route is 0.
The number of NaN's in the column School_Route is 0.
The number of NaN's in the column Average_Trip_Length is 36.
The number of NaN's in the column Operating_Ratio is 36.
The number of NaN's in the column Annual_Passengers is 36.
The number of NaN's in the column Average_Weekday_Passengers is 38.
The number of NaN's in the column Passenger_Miles is 38.
The number of NaN's in the column Passenger_Revenue is 36.


In [63]:
# looking at these specific rows containing NaN's

df_aggregate1[df_aggregate1.isna().any(axis=1)]

,Vehicle_Type,Route,School_Route,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
0,Bus,15B,No,NaN,NaN,NaN,NaN,NaN,NaN
62,Bus,75,No,2.9,41.0,1216962.0,3977.0,NaN,1313990.0
86,Bus,91,No,13.1,12.0,4400.0,NaN,57600.0,6693.0
123,Bus,BLVDDIR,No,NaN,NaN,NaN,NaN,NaN,NaN
124,Bus,312,No,NaN,NaN,NaN,NaN,NaN,NaN
125,Bus,101B,No,NaN,NaN,NaN,NaN,NaN,NaN
130,Bus,33S,Yes,NaN,NaN,NaN,NaN,NaN,NaN
131,Bus,88S,Yes,NaN,NaN,NaN,NaN,NaN,NaN
132,Bus,AFG,Yes,NaN,NaN,NaN,NaN,NaN,NaN
133,Bus,AFL,Yes,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# There are some values that have a lot, so going to filter and save those
df_aggregate1.dropna(thresh=8, inplace=True)

In [65]:
df_aggregate1.reset_index(drop=True)

,Vehicle_Type,Route,School_Route,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
0,Bus,1,No,6.5,17.0,833735.00,2905.0,5419300.0,900209.0
1,Bus,2,No,2.2,25.0,1580796.00,5166.0,3477800.0,1706833.0
2,Bus,3,No,2.1,41.0,2794392.00,9132.0,5868200.0,3017189.0
3,Bus,4,No,2.8,31.0,2223702.00,7267.0,6226400.0,2400998.0
4,Bus,5,No,2.7,24.0,1091502.00,3567.0,2947100.0,1178527.0
5,Bus,6,No,2.0,46.0,2129148.00,6958.0,4258300.0,2298905.0
6,Bus,7,No,2.1,27.0,1654848.00,5408.0,3475200.0,1786789.0
7,Bus,8,No,3.2,31.0,727260.00,2852.0,2327200.0,785244.0
8,Bus,9,No,4.9,23.0,1474614.00,4819.0,7225600.0,1592185.0
9,Bus,12,No,2.4,21.0,766836.00,2506.0,1840400.0,827976.0


In [66]:
for column in df_aggregate1.columns:
    print("The number of NaN's in the column {} is {}.".format(column, df_aggregate1[column].isna().sum()))

The number of NaN's in the column Vehicle_Type is 0.
The number of NaN's in the column Route is 0.
The number of NaN's in the column School_Route is 0.
The number of NaN's in the column Average_Trip_Length is 0.
The number of NaN's in the column Operating_Ratio is 0.
The number of NaN's in the column Annual_Passengers is 0.
The number of NaN's in the column Average_Weekday_Passengers is 2.
The number of NaN's in the column Passenger_Miles is 2.
The number of NaN's in the column Passenger_Revenue is 0.


In [68]:
# Yep, those are the ones I want!
## After checking those NaN rows, those are two pairs of duplicate rows...
df_aggregate1.drop_duplicates(inplace=True)

In [92]:
'''
Interestingly, all rows where School_Route == 'Yes' 
had all NaN values for the rest of the column values in those rows,
leaving only "No" values. Will drop this column as a result...
'''
df_aggregate1.drop(columns='School_Route', inplace=True)
df_aggregate1.reset_index(drop=True)

,Vehicle_Type,Route,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
0,Bus,1,6.5,17.0,833735.00,2905.0,5419300.0,900209.0
1,Bus,2,2.2,25.0,1580796.00,5166.0,3477800.0,1706833.0
2,Bus,3,2.1,41.0,2794392.00,9132.0,5868200.0,3017189.0
3,Bus,4,2.8,31.0,2223702.00,7267.0,6226400.0,2400998.0
4,Bus,5,2.7,24.0,1091502.00,3567.0,2947100.0,1178527.0
5,Bus,6,2.0,46.0,2129148.00,6958.0,4258300.0,2298905.0
6,Bus,7,2.1,27.0,1654848.00,5408.0,3475200.0,1786789.0
7,Bus,8,3.2,31.0,727260.00,2852.0,2327200.0,785244.0
8,Bus,9,4.9,23.0,1474614.00,4819.0,7225600.0,1592185.0
9,Bus,12,2.4,21.0,766836.00,2506.0,1840400.0,827976.0


In [93]:
# Finalizing this dataset

df_Agg = df_aggregate1.copy()

In [119]:
df_Agg.describe()

,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
count,137.000000,137.000000,1.370000e+02,136.000000,1.360000e+02,1.370000e+02
mean,4.405109,27.350365,2.037569e+06,6803.260926,7.303737e+06,2.243535e+06
std,3.839239,11.496541,5.492357e+06,18136.260727,2.474275e+07,5.988550e+06
min,1.100000,8.000000,1.189000e+01,1.486000,1.110000e+02,6.693000e+03
25%,2.400000,19.000000,3.445400e+05,1244.250000,1.346337e+06,4.527880e+05
50%,3.100000,25.000000,9.634400e+05,3540.000000,3.209250e+06,9.912620e+05
75%,5.300000,34.000000,2.129148e+06,7035.250000,6.896500e+06,2.361782e+06
max,32.000000,92.000000,5.476741e+07,180512.000000,2.574068e+08,5.981039e+07


In [122]:
# Not readable...

pd.set_option('display.float_format', '{:.2f}'.format)

In [123]:
df_Agg.describe()

,Average_Trip_Length,Operating_Ratio,Annual_Passengers,Average_Weekday_Passengers,Passenger_Miles,Passenger_Revenue
count,137.00,137.00,137.00,136.00,136.00,137.00
mean,4.41,27.35,2037568.76,6803.26,7303736.65,2243535.42
std,3.84,11.50,5492356.73,18136.26,24742747.01,5988550.49
min,1.10,8.00,11.89,1.49,111.00,6693.00
25%,2.40,19.00,344540.00,1244.25,1346337.00,452788.00
50%,3.10,25.00,963440.00,3540.00,3209250.00,991262.00
75%,5.30,34.00,2129148.00,7035.25,6896500.00,2361782.00
max,32.00,92.00,54767414.00,180512.00,257406800.00,59810392.00


### Aggregate Dataset Description

One downside to SEPTA's open dataset is that it does not have column descriptions. Based on my research, I have garnered the following descriptions. Each value is based on the aggregate compilation of Fiscal Year 2017 for a single particular route of transportation.

(Non-numeric columns are self-explanatory)

- __Average Trip Length:__ Average length in miles a passenger's trip is.
- __Operating Ratio:__ (Unknown)
- __Annual Passengers:__ Total number of passengers taking that route of transportation.
- __Average Weekday Passengers:__ Number of passengers taking that route of transportation for one week. I could not find data specifically on Monday vs. Tuesday vs. Wednesday, etc.
- __Passenger Miles:__ Total miles traveled by all passengers on that route of transportation.
- __Passenger Revenue:__ Total revenue in USD generated from route fare. 
    - With the SEPTA key card, the fare is ___\$2.00.___
    - Without the SEPTA key card (i.e., in person or at a kiosk with cash), paying the fare is ___\$2.50.___

# NEXT STEPS!

- Create revenue x passenger calculator column
- do quick visuals
- draw conclusions
- add real answers

***
#### II. Non-Aggregated Datasets

Taking a look at MFL and BSL Spring 2018 datasets, we see we have a lot of irrelevant information such as "bench", "recycling bin", "Philadelphia City Council District", and other similar features. Admittedly, we would not leave any data behind without first confirming through visualization and related techniques that those features are, indeed, irrelevant. However, in the spirit of keeping this analysis concise, I will intentionally put the cart before the horse and delete those features I find to have zero explanatory power for our target variable, SEPTA key taps.

In [100]:
df_MFL.columns

Index(['X', 'Y', 'OBJECTID', 'Route', 'Station', 'Stop_ID', 'On_Street',
       'At_Street', 'Street_Address', 'City', 'State', 'Zip', 'County',
       'Municipality', 'Owner', 'Average_Weekday_Ridership',
       'Average_Saturday_Ridership', 'Average_Sunday_Ridership',
       'Ticket_Office', 'Handicap_Accesible', 'Parking_Available',
       'Number_of_Daily_Parking_Spaces', 'Number_of_Permit_Parking_Spaces',
       'Taxi_Stand', 'Platform_Type', 'Station_Building', 'Shelter', 'Bench',
       'Elevator', 'Escaltor', 'Lighting', 'Restroom', 'Trash_Receptacle',
       'Recycling_Bin', 'Map', 'Fare_Information', 'Schedule_Information',
       'Minority_Area', 'Low_Income_Area', 'Minority___Low_Income_Area',
       'PA_State_Senate_District', 'PA_State_House_District',
       'PA_U_S__Congressional_District', 'Philadelphia_City_Council_Distr',
       'Latitude', 'Longitude', 'GlobalID'],
      dtype='object')

In [117]:
df_MFL[['Route', 'Average_Weekday_Ridership',
       'Average_Saturday_Ridership', 'Average_Sunday_Ridership',
       'Minority_Area', 'Low_Income_Area', 'Minority___Low_Income_Area']].head()

,Route,Average_Weekday_Ridership,Average_Saturday_Ridership,Average_Sunday_Ridership,Minority_Area,Low_Income_Area,Minority___Low_Income_Area
0,Market-Frankford Line,15439.292490,8679.725490,6608.436364,Yes,Yes,Yes
1,Market-Frankford Line,427.426877,297.431373,229.290909,Yes,Yes,Yes
2,Market-Frankford Line,1953.189723,1218.901961,931.763636,Yes,Yes,Yes
3,Market-Frankford Line,4746.288538,2781.254902,2033.127273,Yes,Yes,Yes
4,Market-Frankford Line,5451.470356,3087.764706,2392.381818,Yes,Yes,Yes


In [101]:
# trolley and bus data has fewer features, but still loads of irrelevant data

df_trolley.columns

Index(['X', 'Y', 'OBJECTID', 'GIS_DB_ID', 'Sign_Up', 'Route', 'Direction',
       'Sequence', 'Stop_ID', 'Stop_Abbreviation', 'Stop_Name', 'Mode',
       'Time_Point', 'Time_Point_ID', 'Time_Point_Abbreviation', 'School',
       'Source', 'Comments', 'Weekday_Boards', 'Weekday_Leaves',
       'Weekday_Total', 'Saturday_Boards', 'Saturday_Leaves', 'Saturday_Total',
       'Sunday_Boards', 'Sunday_Leaves', 'Sunday_Total', 'Latitude',
       'Longitude', 'GlobalID'],
      dtype='object')

In [116]:
df_trolley[['Route', 'Mode','Stop_ID','School', 'Weekday_Boards', 'Weekday_Leaves', 'Weekday_Total',
            'Saturday_Boards', 'Saturday_Leaves', 'Saturday_Total',
            'Sunday_Boards', 'Sunday_Leaves', 'Sunday_Total']].head()

,Route,Mode,Stop_ID,School,Weekday_Boards,Weekday_Leaves,Weekday_Total,Saturday_Boards,Saturday_Leaves,Saturday_Total,Sunday_Boards,Sunday_Leaves,Sunday_Total
0,11,Trolley,305,No,1360.0,0.0,1360.0,482.0,0.0,482.0,298.0,0.0,298.0
1,11,Trolley,24568,No,43.0,6.0,49.0,21.0,2.0,23.0,12.0,0.0,12.0
2,11,Trolley,20697,No,40.0,9.0,49.0,13.0,3.0,16.0,4.0,1.0,5.0
3,11,Trolley,20698,No,85.0,31.0,116.0,28.0,11.0,39.0,18.0,8.0,26.0
4,11,Trolley,20699,No,163.0,29.0,192.0,45.0,12.0,57.0,52.0,7.0,59.0


In [113]:
totalboards2018_trolley = df_trolley.Weekday_Total.sum() + df_trolley.Saturday_Total.sum() + df_trolley.Sunday_Total.sum()
totalboards2018_trolley

187473.0

In [115]:
df_trolley[['Route', 'Stop_ID','School', 'Weekday_Boards', 'Weekday_Leaves', 'Weekday_Total',
            'Saturday_Boards', 'Saturday_Leaves', 'Saturday_Total',
            'Sunday_Boards', 'Sunday_Leaves', 'Sunday_Total']].sum(numeric_only=True)

Route                 23023.0
Stop_ID            12508972.0
Weekday_Boards        62718.0
Weekday_Leaves        62622.0
Weekday_Total        125340.0
Saturday_Boards       18101.0
Saturday_Leaves       17894.0
Saturday_Total        35995.0
Sunday_Boards         13170.0
Sunday_Leaves         12968.0
Sunday_Total          26138.0
dtype: float64

In [111]:
type(df_trolley.Weekday_Total[1])

numpy.float64

In [112]:
df_trolley.describe()

,X,Y,OBJECTID,Route,Sequence,Stop_ID,Time_Point_ID,Weekday_Boards,Weekday_Leaves,Weekday_Total,Saturday_Boards,Saturday_Leaves,Saturday_Total,Sunday_Boards,Sunday_Leaves,Sunday_Total,Latitude,Longitude
count,680.000000,680.000000,680.000000,680.000000,680.000000,680.000000,648.000000,617.000000,617.000000,617.000000,470.000000,470.00000,470.000000,472.000000,472.000000,472.000000,680.000000,680.000000
mean,-75.225779,39.946850,340.500000,33.857353,23.277941,18395.547059,94.993827,101.649919,101.494327,203.144246,38.512766,38.07234,76.585106,27.902542,27.474576,55.377119,39.946850,-75.225779
std,0.053672,0.020273,196.443376,32.998085,14.516278,7065.045046,304.203330,301.916412,219.204547,390.665861,110.990918,85.56255,144.182661,80.544795,59.167144,102.395624,0.020273,0.053672
min,-75.393572,39.898668,1.000000,10.000000,1.000000,277.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,39.898668,-75.393572
25%,-75.246232,39.928987,170.750000,11.000000,11.000000,18843.750000,0.000000,11.000000,11.000000,38.000000,4.000000,4.00000,13.000000,2.000000,2.000000,10.000000,39.928987,-75.246232
50%,-75.223803,39.948519,340.500000,15.000000,22.000000,20741.500000,0.000000,34.000000,34.000000,95.000000,12.000000,12.50000,39.500000,8.000000,9.000000,26.000000,39.948519,-75.223803
75%,-75.197306,39.964106,510.250000,36.000000,34.000000,20931.250000,0.000000,104.000000,112.000000,212.000000,38.000000,40.00000,76.750000,28.000000,28.250000,57.250000,39.964106,-75.197306
max,-75.099516,39.984206,680.000000,102.000000,62.000000,31540.000000,2186.000000,3897.000000,2264.000000,3898.000000,1429.000000,912.00000,1429.000000,938.000000,620.000000,938.000000,39.984206,-75.099516


In [ ]:
df_BSL
df_MFL
df_Norris
df_bus2018
df_bus2019
df_trolley

 2 dollars per ride Norris/BSL/MFL/Bus vs. $2.50 cash fare
https://www.iseptaphilly.com/key

***
## 3. What Target Metric Should be Used for Calculating SEPTA Key Taps?

If there was a data series of "SEPTA Card Key Taps", the analysis would be fairly straightforward. Upon review of the data (as taken from above), there is no such data series.  Therefore, we will have to construct our own translation between the target variable (number of key taps) and the available features.

While feature exploration/selection is always a critical part of data analysis, a good starting point is to look more into _ridership_ and _revenue_. 

***

***
## 4. Data Analysis

Now that we 

http://septa.org/strategic-plan/pdf/2019-11-revenue-ride.pdf

https://tabula-py.readthedocs.io/en/latest/getting_started.html#get-tabula-py-working-windows-10

In [46]:
import tabula

In [48]:
df2019ridership = tabula.read_pdf('http://septa.org/strategic-plan/pdf/2019-11-revenue-ride.pdf',
                                 pages='all')

JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

In [18]:
df = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA_Market_Frankford_Line.csv')

In [19]:
df.head()

,OBJECTID,Route,Vehicle_Hours,Vehicle_Miles,Peak_Vehicles,Average_Weekday_Passengers,Annual_Passengers,Average_Trip_Length,Passenger_Miles,Passenger_Revenue,Variable_Expenses,Variable_Recovery,Fully_Allocated_Expenses,Operating_Ratio,GlobalID,Shape__Length
0,1,Market-Frankford Line,470930,9230221,144,180512,54767414,4.7,257406800,59810392,137752084,158,136694484,44,9b7ec014-18d2-42af-a482-a86be8a370e6,0.237995
